# Import Modul

In [1]:
import pandas as pd
from Nawa import nawa
from Nawa import ngram
#import re

In [3]:
#Kata Negasi (bisa di-update)
kata_negasi = ["tidak", "bukan","tiada","tak", "jangan"]

# Load Jawaban

In [3]:
data = pd.read_excel("data/jawaban_clean.xlsx")
jawaban_ = data['jawaban_tanpa_stem'].tolist()
score = data['score']
data.head()
#jawaban_ = jawaban_[311:314]

,Unnamed: 0,jawaban_tanpa_stem,jawaban,score
0,0,biro perjalanan buana wisata menawarkan berbag...,biro jalan buana wisata tawar bagai jalan wisa...,1
1,1,jangan lewatkan kesempatan anda untuk mengetah...,jangan lewat sempat tahu cara langsung machu p...,1
2,2,dengan mengaksesnya atau sourching melalui int...,akses sourching lalu internet dengan tanya lan...,0
3,3,biro perjalanan buana wisata menawarkan berbag...,biro jalan buana wisata tawar bagai jalan wisa...,0
4,4,menanyakan kepada orang yang tinggal di tempat...,tanya orang tinggal tempat machu picchu dapat ...,0


In [44]:
data = pd.read_excel("data/hasil8.xlsx")
data = data[:100]
#jawaban_ = data['jawaban_tanpa_stem'].tolist()
jawaban_ = data['jawaban'].tolist()
#score = data['score']
data.head()

,Column1,jawaban,jawaban_cektypo,tf_idf_jaccard,tf_idf_cosine,tf_idf_dice,tf_idf_df_jaccard,tf_idf_df_cosine,tf_idf_df_dice,df_jaccard,df_cosine,df_dice
0,753,telepon ke 021 7381 1111 untuk informasi lebih...,telepon ke 021 7381 1111 untuk informasi lebih...,1.0,1.0,1.0,0.666667,1.0,0.8,0.666667,1.0,0.8
1,818,telepon ke 021 7381 1111 sekarang juga untuk i...,telepon ke 021 7381 1111 sekarang juga untuk i...,1.0,1.0,1.0,0.666667,1.0,0.8,0.666667,1.0,0.8
2,820,telepon ke 0217381 1111 untuk informasi lebih ...,telepon ke 0217381 1111 untuk informasi lebih ...,1.0,1.0,1.0,0.666667,1.0,0.8,0.666667,1.0,0.8
3,883,telepon ke no 02173811111 untuk informasi lebi...,telepon ke no 02173811111 untuk informasi lebi...,1.0,1.0,1.0,0.666667,1.0,0.8,0.666667,1.0,0.8
4,1095,telepone 0217381 1111 sekarang juga untuk info...,telepon 0217381 1111 sekarang juga untuk infor...,1.0,1.0,1.0,0.666667,1.0,0.8,0.666667,1.0,0.8


# Input kunci jawaban

In [91]:
kunci_jawaban = ["telepon 02173811111 buanawisata.com"]
jawaban = "tidakk kunjunngi 1telepon0217381 1111 2 kunjungi website kami di buanawisata.com"

# Implementasi

# Praproses

In [92]:

def cek_negasi_list(kunci_jawaban):
    if type(kunci_jawaban) is not list:
        kunci_jawaban = [kunci_jawaban]
    new_kj = list()
    for i in kunci_jawaban:
        i = nawa.ubah_simbol(i)
        hasil = nawa.cek_negasi(kata_negasi,i)
        new_kj.append(hasil)
    return new_kj
#cek_negasi_list("tidak telepon pada 02173811111 website buanawisata.com")

def get_unik(kunci_jawaban):
    if type(kunci_jawaban) is not list:
        kunci_jawaban = [kunci_jawaban]
    list_kata_kj = list()
    for i in kunci_jawaban:
        i = nawa.ubah_simbol(i)
        for j in i.split():
            list_kata_kj.append(j)
    list_kata_kj = list(set(list_kata_kj))
    return list_kata_kj
nega = cek_negasi_list(kunci_jawaban)
fitur = get_unik(nega)

kunci_jawaban_unik = " ".join(get_unik(kunci_jawaban))
def praproses(jawaban, kunci_jawaban_unik):
    jawaban = nawa.ubah_simbol(jawaban)
    jawaban = nawa.pisahKata(kunci_jawaban_unik, jawaban)
    jawaban_ngram = ngram.en_geram(kunci_jawaban_unik, jawaban).split()
    #jawaban = jawaban +" "+ " ".join(list(set(jawaban_ngram.split())))
    jawaban = jawaban.split()
    for i in jawaban_ngram:
        if i not in jawaban:
            jawaban.append(i)
    jawaban = " ".join(jawaban)
    jawaban = nawa.cek_typo(kunci_jawaban_unik, jawaban, 0.95)
    jawaban = nawa.cek_negasi(kata_negasi, jawaban)
    return jawaban

praproses("buanawisata.com", kunci_jawaban_unik)


'buanawisatacom'

'nomor mengunjungi buanawisata.com 02173811111 di menelepon bisa website'

## Persiapan data

In [68]:

jawaban_praproses = praproses(jawaban, kunci_jawaban_unik)
kunci_jawaban_negasi_list = cek_negasi_list(kunci_jawaban)
nega = cek_negasi_list(kunci_jawaban)
fitur = get_unik(nega)
print("fitur         ->",fitur)
print("Kunci Jawaban ->",kunci_jawaban_negasi_list)
print("Jawaban       ->",jawaban_praproses)

fitur         -> ['mengunjungi', '02173811111', 'buanawisatacom', 'menelepon', 'website']
Kunci Jawaban -> ['menelepon 02173811111 mengunjungi website buanawisatacom']
Jawaban       -> tidak_ kunjunngi 1telepon0217381 1111 2 kunjungi website kami di buanawisatacom


# Menghitung Similarity (tf-idf-df)

In [95]:
def essay_similarity_jaccard(kunci_jawaban_negasi_list, jawaban):
    if type(kunci_jawaban_negasi_list) != list:
        kunci_jawaban_negasi_list = [kunci_jawaban_negasi_list]
    simm = list()   
    for kj in kunci_jawaban_negasi_list:
        t = nawa.tf_idf_df(kj, jawaban, fitur)
        jaccard = nawa.jaccard_baru(t.A[0], t.A[1])
        simm.append(jaccard)
    return max(simm)


def essay_similarity_cosine(kunci_jawaban_negasi_list, jawaban):
    if type(kunci_jawaban_negasi_list) != list:
        kunci_jawaban_negasi_list = [kunci_jawaban_negasi_list]
    simm = list()   
    for kj in kunci_jawaban_negasi_list:
        t = nawa.tf_idf_df(kj, jawaban, fitur)
        jaccard = nawa.cosine_similarity(t.A[0], t.A[1])
        simm.append(jaccard)
    return max(simm)

def essay_similarity_dice(kunci_jawaban_negasi_list, jawaban):
    if type(kunci_jawaban_negasi_list) != list:
        kunci_jawaban_negasi_list = [kunci_jawaban_negasi_list]
    simm = list()   
    for kj in kunci_jawaban_negasi_list:
        t = nawa.tf_idf_df(kj, jawaban, fitur)
        jaccard = nawa.dice_similarity(t.A[0], t.A[1])
        simm.append(jaccard)
    return max(simm)


def transform(simmm):
    if simmm <= 0.25:
        return "sangat tidak mirip"
    elif simmm <= 0.5:
        return "tidak mirip"
    elif simmm <= 0.75:
        return "mirip"
    elif simmm <= 1:
        return "sangat mirip"
    else:
        return "error"

## Proses Perhitungan Similarity

In [96]:
dice_list = list()
jaccard_list = list()
cosine_list = list()

dice_list_t = list()
jaccard_list_t = list()
cosine_list_t = list()

for i in jawaban_:
    jawaban_praproses = praproses(i, kunci_jawaban_unik)
    #print(jawaban_praproses)
    dice_ = essay_similarity_dice(kunci_jawaban_negasi_list, i)
    jaccard_ = essay_similarity_cosine(kunci_jawaban_negasi_list, i)
    cosine_ = essay_similarity_jaccard(kunci_jawaban_negasi_list, i)
    
    dice_list.append(dice_)
    cosine_list.append(jaccard_)
    jaccard_list.append(cosine_)
    
    dice_list_t.append(transform(dice_))
    cosine_list_t.append(transform(jaccard_))
    jaccard_list_t.append(transform(cosine_))

In [97]:
dice_list

[0.0,
 0.0,
 0.0,
 0.33609692727625745,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.33609692727625745,
 0.0,
 0.33609692727625745,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.33609692727625745,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.33609692727625745,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.33609692727625745,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.33609692727625745,
 0.33609692727625745,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.5797386715376657,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.33609692727625745,
 0.5797386715376657,
 0.0,
 0.0,
 0.33609692727625745,
 0.0,
 0.0,
 0.0,
 0.0,
 0.33609692727625745,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.33609692727625745,
 0.0,
 0.0]

In [90]:
for i in jawaban_:
    print(praproses(i, kunci_jawaban_unik))
    print("")

telepon ke 021 7381 1111 untuk informasi lebih lanjut kunjungi website di buanawisatacom untuk informasi paket dan harga 02173811111

telepon ke 021 7381 1111 sekarang juga untuk informasi lebih lanjut atau kunjungi website kami di buanawisatacom untuk informasi paket dan harga 02173811111

telepon ke 0217381 1111 untuk informasi lebih lanjut atau kunjungi website kami di buanawisatacom untuk informasi paket dan harga 02173811111

telepon ke no 02173811111 untuk informasi lebih lanjut dan kunjungi website mereka di buanawisatacom untuk informasi paket dan harga

telepone 0217381 1111 sekarang juga untuk informasi lebih lanjut kunjungi website kami di buanawisatacom untuk informasi dan harga 02173811111

telepon ke nomor 0217381 1111 yg merupakan nomor kantor biro perjalanan buana wisata mengunjungi website buanawisatacom 02173811111

telepon = 021 7381 1111 website = buanawisatacom 02173811111

telepon ke 021 7381 1111 kunjungi website kami di buanawisatacom untuk informasi paket dan h

telepon 021 7381 1111 buanawisatacom 02173811111

menghubungi telepon 02173811111 kunjungi websait di buanawisatacom

dengan cara menghubungi nmomor telepon 0217381 1111 dengan mengunjungi website buanawisatacom 02173811111

untuk memesan paket wisata biro kita dapat menghubungi nomor telepon 0217381 1111 untuk informasi lebih lengkap untuk memesan paket wisata anda dapat mengunjungi website buanawisatacom 02173811111

telepon 0217381 1111 kunjungi website di buanawisatacom 02173811111

telepon 021 7381 1111 kunjungi website kami di buanawisatacom 02173811111

telepon 0217381 1111 kunjungi website di buanawisatacom 02173811111

untuk mengetahui secara langsung pengujung dapat menghubungi atau telepon 02173811111 untuk mengetahui secara langsung pengujung juga dapat mengunjungi website di buanawisatacom

telepon 0217381 1111 sekarang juga untuk informasi lebih lanjut kunjungi website kami di buanawisatacom untuk informasi paket dan harga 02173811111

telepon 021 7381 1111 kunjungi websi

## Implementasi

In [98]:
kunci_jawaban = nawa.ubah_simbol("telepon 02173811111 buanawisata.com")
#mmmkunci_jawaban = nawa.cek_negasi(kata_negasi, kunci_jawaban)
fitur = nawa.cek_negasi(kata_negasi, kunci_jawaban).split()
#jawaban = ["'dengan jalan kaki naik kreta'", "fsdjifjsd jfsdjf k klfjsd kjsklj lfsdj","fsdjifjsd jfsdjf k klfjsd kjsklj lfsdj"]
#jawaban = jawaban
#tf_idf_df_
jawaban_cektypo = []
tf_idf_jaccard = []
tf_idf_cosine = []
tf_idf_dice = []

tf_idf_df_jaccard = []
tf_idf_df_cosine = []
tf_idf_df_dice = []

df_jaccard = []
df_cosine = []
df_dice = []
ct =0
for i in jawaban_:
    #if len(i)<=2:
    #print(ct)
    if type(i)is float:
        jawaban_cektypo.append("")
        tf_idf_jaccard.append(0.0)
        tf_idf_cosine.append(0.0)
        tf_idf_dice.append(0.0)

        tf_idf_df_jaccard.append(0.0)
        tf_idf_df_cosine.append(0.0)
        tf_idf_df_dice.append(0.0)

        df_jaccard.append(0.0)
        df_cosine.append(0.0)
        df_dice.append(0.0)
    #if ct > 5:
        #break
    #if type(i)is float:
        #print(i)
        #i = "#"
        #print(i)
    #print(type(i))
    #print(ct)
    #print(i)
    
    
    #if len(i) <= 1:
        #i = "# #"
        #print(i)
    else:
        jawaban = nawa.ubah_simbol(i)
        jawaban_cektypo.append(nawa.cek_typo(kunci_jawaban, jawaban, 0.97))
        jawaban = nawa.pisahKata(kunci_jawaban, jawaban)
        jawaban_ngram = ngram.en_geram(kunci_jawaban, jawaban)
        jawaban = jawaban +" "+jawaban_ngram
        jawaban = nawa.cek_typo(kunci_jawaban, jawaban, 0.95)
        jawaban = nawa.cek_negasi(kata_negasi, jawaban)
        jawaban_ct= jawaban
    
        t = nawa.tf_idf(kunci_jawaban, jawaban_ct, fitur)
        jaccard = nawa.jaccard_baru(t.A[0], t.A[1])
        cosine = nawa.cosine_similarity(t.A[0], t.A[1])
        dice = nawa.dice_similarity(t.A[0], t.A[1])
        tf_idf_jaccard.append(jaccard)
        tf_idf_cosine.append(cosine)
        tf_idf_dice.append(dice)
          
        t = nawa.tf_idf_df(kunci_jawaban, jawaban_ct, fitur)
        jaccard = nawa.jaccard_baru(t.A[0], t.A[1])
        cosine = nawa.cosine_similarity(t.A[0], t.A[1])
        dice = nawa.dice_similarity(t.A[0], t.A[1])
        tf_idf_df_jaccard.append(jaccard)
        tf_idf_df_cosine.append(cosine)
        tf_idf_df_dice.append(dice)
    
        t = nawa.df(kunci_jawaban, jawaban_ct, fitur)
        jaccard = nawa.jaccard_baru(t.A[0], t.A[1])
        cosine = nawa.cosine_similarity(t.A[0], t.A[1])
        dice = nawa.dice_similarity(t.A[0], t.A[1])
        df_jaccard.append(jaccard)
        df_cosine.append(cosine)
        df_dice.append(dice)
    ct+=1
    
    
dict = {"jawaban":jawaban_, 
        "jawaban_cektypo":jawaban_cektypo,
        "tf_idf_jaccard":tf_idf_jaccard, 
        "tf_idf_cosine":tf_idf_cosine,
        "tf_idf_dice":tf_idf_dice,
       "tf_idf_df_jaccard":tf_idf_df_jaccard,
        "tf_idf_df_cosine":tf_idf_df_cosine,
        "tf_idf_df_dice":tf_idf_df_dice,
        "df_jaccard":df_jaccard,
        "df_cosine":df_cosine,
        "df_dice":df_dice
       }
df = pd.DataFrame(dict, columns=["jawaban", "jawaban_cektypo","tf_idf_jaccard", "tf_idf_cosine","tf_idf_dice","tf_idf_df_jaccard","tf_idf_df_cosine","tf_idf_df_dice","df_jaccard","df_cosine","df_dice"])                      

In [100]:
import datetime
writer = pd.ExcelWriter('data/hasil_10.xlsx', engine='xlsxwriter')
df.to_excel(writer, sheet_name='Sheet1')
# Close the Pandas Excel writer and output the Excel file.
writer.save()
df

,jawaban,jawaban_cektypo,tf_idf_jaccard,tf_idf_cosine,tf_idf_dice,tf_idf_df_jaccard,tf_idf_df_cosine,tf_idf_df_dice,df_jaccard,df_cosine,df_dice
0,telepon ke 021 7381 1111 untuk informasi lebih...,telepon ke 021 7381 1111 untuk informasi lebih...,1.0,1.0,1.0,0.666667,1.0,0.8,0.666667,1.0,0.8
1,telepon ke 021 7381 1111 sekarang juga untuk i...,telepon ke 021 7381 1111 sekarang juga untuk i...,1.0,1.0,1.0,0.666667,1.0,0.8,0.666667,1.0,0.8
2,telepon ke 0217381 1111 untuk informasi lebih ...,telepon ke 0217381 1111 untuk informasi lebih ...,1.0,1.0,1.0,0.666667,1.0,0.8,0.666667,1.0,0.8
3,telepon ke no 02173811111 untuk informasi lebi...,telepon ke no 02173811111 untuk informasi lebi...,1.0,1.0,1.0,0.666667,1.0,0.8,0.666667,1.0,0.8
4,telepone 0217381 1111 sekarang juga untuk info...,telepon 0217381 1111 sekarang juga untuk infor...,1.0,1.0,1.0,0.666667,1.0,0.8,0.666667,1.0,0.8
5,telepon ke nomor 0217381 1111 yg merupakan nom...,telepon ke nomor 0217381 1111 yg merupakan nom...,1.0,1.0,1.0,0.666667,1.0,0.8,0.666667,1.0,0.8
6,telepon = 021 7381 1111 website = buanawisata.com,telepon = 021 7381 1111 website = buanawisatacom,1.0,1.0,1.0,0.666667,1.0,0.8,0.666667,1.0,0.8
7,telepon ke 021 7381 1111 kunjungi website kami...,telepon ke 021 7381 1111 kunjungi website kami...,1.0,1.0,1.0,0.666667,1.0,0.8,0.666667,1.0,0.8
8,telepon ke 021 7381 1111 kunjungi website buan...,telepon ke 021 7381 1111 kunjungi website buan...,1.0,1.0,1.0,0.666667,1.0,0.8,0.666667,1.0,0.8
9,telepon ke 0217381 1111 untuk informasi lebih ...,telepon ke 0217381 1111 untuk informasi lebih ...,1.0,1.0,1.0,0.666667,1.0,0.8,0.666667,1.0,0.8
